To add retrieval scores to results from retrievers in LangChain, you can follow these steps based on the type of retriever:

# 1. Adding Scores to Vector Store Retriever Results
For a vector store retriever (e.g., PineconeVectorStore), use a wrapper function around similarity_search_with_score to include similarity scores in each document's metadata. Here’s an example:

In [1]:
from langchain_core.documents import Document
from langchain_core.runnables import chain
from typing import List

@chain
def retriever(query: str) -> List[Document]:
    docs, scores = zip(*vectorstore.similarity_search_with_score(query))
    for doc, score in zip(docs, scores):
        doc.metadata["score"] = score
    return docs


# 2. Adding Scores in SelfQueryRetriever
The SelfQueryRetriever uses an LLM to generate a query, which can also filter results based on metadata. To propagate similarity scores, subclass SelfQueryRetriever and override its _get_docs_with_query method:

In [2]:
from typing import Any, Dict
from langchain.retrievers.self_query.base import SelfQueryRetriever

class CustomSelfQueryRetriever(SelfQueryRetriever):
    def _get_docs_with_query(
        self, query: str, search_kwargs: Dict[str, Any]
    ) -> List[Document]:
        docs, scores = zip(*self.vectorstore.similarity_search_with_score(query, **search_kwargs))
        for doc, score in zip(docs, scores):
            doc.metadata["score"] = score
        return docs


# 3. Adding Scores in MultiVectorRetriever
For a MultiVectorRetriever, which handles documents with multiple vectors (e.g., chunks of a larger document), subclass MultiVectorRetriever and override _get_relevant_documents. This allows you to:

Add scores to each sub-document’s metadata.
Include relevant sub-documents with their scores in the parent document’s metadata.

# 

In [3]:
from collections import defaultdict
from langchain.retrievers import MultiVectorRetriever

class CustomMultiVectorRetriever(MultiVectorRetriever):
    def _get_relevant_documents(
        self, query: str, *, run_manager
    ) -> List[Document]:
        results = self.vectorstore.similarity_search_with_score(query, **self.search_kwargs)
        id_to_doc = defaultdict(list)
        
        for doc, score in results:
            doc_id = doc.metadata.get("doc_id")
            if doc_id:
                doc.metadata["score"] = score
                id_to_doc[doc_id].append(doc)
        
        docs = []
        for _id, sub_docs in id_to_doc.items():
            docstore_docs = self.docstore.mget([_id])
            if docstore_docs:
                if doc := docstore_docs[0]:
                    doc.metadata["sub_docs"] = sub_docs
                    docs.append(doc)
        
        return docs
